In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
import penaltyblog as pb
from scipy.optimize import minimize
from scipy.stats import poisson

In [3]:
df = pb.footballdata.fetch_data("England", 2018, 0)
df[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]].head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG
0,2018-08-10,Man United,Leicester,2,1
1,2018-08-11,Bournemouth,Cardiff,2,0
2,2018-08-11,Fulham,Crystal Palace,0,2
3,2018-08-11,Huddersfield,Chelsea,0,3
4,2018-08-11,Newcastle,Tottenham,1,2


### Poisson Model

In [4]:
pois = pb.poisson.PoissonGoalsModel(
    df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"]
)

In [5]:
%%timeit -n 1 -r 1
pois.fit()

12.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
pois

Module: Penaltyblog

Model: Poisson

Number of parameters: 41
Log Likelihood: -1065.077
AIC: 2212.154

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.362                -0.856              
Bournemouth          1.115                -0.555              
Brighton             0.634                -0.731              
Burnley              0.894                -0.595              
Cardiff              0.614                -0.592              
Chelsea              1.202                -1.135              
Crystal Palace       1.004                -0.839              
Everton              1.055                -0.978              
Fulham               0.626                -0.431              
Huddersfield         0.184                -0.507              
Leicester            0.999                -0.939              
Liverpool            1.532                -1.683              
Man City         

In [7]:
pois.get_params()

{'attack_Arsenal': 1.3621745496246604,
 'attack_Bournemouth': 1.115185287448802,
 'attack_Brighton': 0.6339106634211633,
 'attack_Burnley': 0.8937361704562544,
 'attack_Cardiff': 0.6136874108839112,
 'attack_Chelsea': 1.2024715923923994,
 'attack_Crystal Palace': 1.0043997814678136,
 'attack_Everton': 1.054819293459089,
 'attack_Fulham': 0.6255680483307611,
 'attack_Huddersfield': 0.1843965301887161,
 'attack_Leicester': 0.9994828625132763,
 'attack_Liverpool': 1.5319631797676319,
 'attack_Man City': 1.598332346974268,
 'attack_Man United': 1.248697804338311,
 'attack_Newcastle': 0.8049189177444026,
 'attack_Southampton': 0.890756819929392,
 'attack_Tottenham': 1.2641739492035766,
 'attack_Watford': 1.0298067450193877,
 'attack_West Ham': 1.0258464326712924,
 'attack_Wolves': 0.9156716141500305,
 'defence_Arsenal': -0.8560763214410562,
 'defence_Bournemouth': -0.5553098917626262,
 'defence_Brighton': -0.730743005878916,
 'defence_Burnley': -0.595420417769868,
 'defence_Cardiff': -0.591

In [8]:
goals = pois.predict("Watford", "Wolves")
goals

Module: Penaltyblog

Class: FootballProbabilityGrid

Home Goal Expectation: 1.3090925091085217
Away Goal Expectation: 1.202994077835413

Home Win: 0.3908094738965409
Draw: 0.26882054755628126
Away Win: 0.3403699785304319

In [9]:
goals.home_draw_away

[0.3908094738965409, 0.26882054755628126, 0.3403699785304319]

### Dixon and Coles

In [10]:
dc = pb.poisson.DixonColesGoalModel(
    df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"]
)

In [11]:
%%timeit -n 1 -r 1
dc.fit()

18.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
dc

Module: Penaltyblog

Model: Dixon and Coles

Number of parameters: 42
Log Likelihood: -1064.943
AIC: 2213.886

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.36                 -0.858              
Bournemouth          1.115                -0.555              
Brighton             0.632                -0.733              
Burnley              0.897                -0.592              
Cardiff              0.615                -0.591              
Chelsea              1.205                -1.13               
Crystal Palace       1.007                -0.837              
Everton              1.054                -0.977              
Fulham               0.625                -0.433              
Huddersfield         0.18                 -0.507              
Leicester            0.996                -0.94               
Liverpool            1.534                -1.679              
Man City 

In [13]:
dc.get_params()

{'attack_Arsenal': 1.3601847051449685,
 'attack_Bournemouth': 1.1145184817774574,
 'attack_Brighton': 0.6322117765690474,
 'attack_Burnley': 0.8970666604876617,
 'attack_Cardiff': 0.6147208002064647,
 'attack_Chelsea': 1.2053671601802987,
 'attack_Crystal Palace': 1.0066731079751312,
 'attack_Everton': 1.0541811871458513,
 'attack_Fulham': 0.6247167441160756,
 'attack_Huddersfield': 0.1802366555401616,
 'attack_Leicester': 0.9964556127567427,
 'attack_Liverpool': 1.533777769321428,
 'attack_Man City': 1.5986946689807402,
 'attack_Man United': 1.2510850286831325,
 'attack_Newcastle': 0.8064977155628419,
 'attack_Southampton': 0.8965791803986825,
 'attack_Tottenham': 1.2590221553622505,
 'attack_Watford': 1.030579198194093,
 'attack_West Ham': 1.023452448569123,
 'attack_Wolves': 0.9139789430224474,
 'defence_Arsenal': -0.8579414650270959,
 'defence_Bournemouth': -0.5551972901001498,
 'defence_Brighton': -0.7332831080243792,
 'defence_Burnley': -0.5924349227742409,
 'defence_Cardiff': -0

In [14]:
goals = dc.predict("Watford", "Wolves")
goals

Module: Penaltyblog

Class: FootballProbabilityGrid

Home Goal Expectation: 1.3058058355892344
Away Goal Expectation: 1.2028436672174572

Home Win: 0.3847266083053733
Draw: 0.2795257473371696
Away Win: 0.3357476443411532

In [15]:
goals.home_draw_away

[0.3847266083053733, 0.2795257473371696, 0.3357476443411532]

### Rue Salvesen Model

In [16]:
rs = pb.poisson.RueSalvesenGoalModel(
    df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"]
)

In [17]:
%%timeit -n 1 -r 1
rs.fit()

28 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
rs

Module: Penaltyblog

Model: Rue Salvesen

Number of parameters: 43
Log Likelihood: -1061.167
AIC: 2208.334

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.452                -0.943              
Bournemouth          1.219                -0.654              
Brighton             0.584                -0.68               
Burnley              0.943                -0.633              
Cardiff              0.596                -0.566              
Chelsea              1.191                -1.137              
Crystal Palace       1.021                -0.846              
Everton              1.041                -0.982              
Fulham               0.645                -0.447              
Huddersfield         0.076                -0.412              
Leicester            0.985                -0.923              
Liverpool            1.476                -1.615              
Man City    

In [19]:
rs.get_params()

{'attack_Arsenal': 1.4515499652333708,
 'attack_Bournemouth': 1.2191251425992289,
 'attack_Brighton': 0.5843791338523971,
 'attack_Burnley': 0.9428394150636694,
 'attack_Cardiff': 0.5956849400952619,
 'attack_Chelsea': 1.1908298214241264,
 'attack_Crystal Palace': 1.0212979883565116,
 'attack_Everton': 1.04093244398572,
 'attack_Fulham': 0.6445841286347946,
 'attack_Huddersfield': 0.07606195171717726,
 'attack_Leicester': 0.9849862932464387,
 'attack_Liverpool': 1.4755351937422723,
 'attack_Man City': 1.5247245284887485,
 'attack_Man United': 1.3290620603598147,
 'attack_Newcastle': 0.7496897900593501,
 'attack_Southampton': 0.9270992466535436,
 'attack_Tottenham': 1.2439656368821477,
 'attack_Watford': 1.0755744438420514,
 'attack_West Ham': 1.0497875097666782,
 'attack_Wolves': 0.8722903659934693,
 'defence_Arsenal': -0.9434890990774417,
 'defence_Bournemouth': -0.6539566991571978,
 'defence_Brighton': -0.6797746720168024,
 'defence_Burnley': -0.6325068403959166,
 'defence_Cardiff': 

In [20]:
probs = rs.predict("Watford", "Wolves")
probs

Module: Penaltyblog

Class: FootballProbabilityGrid

Home Goal Expectation: 1.3094395353322117
Away Goal Expectation: 1.2096312592832934

Home Win: 0.38433731314981423
Draw: 0.27872007475335486
Away Win: 0.33694261207972276

In [21]:
goals.home_draw_away

[0.3847266083053733, 0.2795257473371696, 0.3357476443411532]

### Market Probabilities

In [22]:
probs = rs.predict("Watford", "Wolves")

In [23]:
probs.total_goals("over", 2.5)

0.46106942905949866

In [24]:
probs.total_goals("under", 2.5)

0.5389305709233934

In [25]:
probs.asian_handicap("away", -1.5)

0.8246939511562793

In [26]:
probs.asian_handicap("home", 1.5)

0.1753060488266135

In [27]:
probs.home_draw_away

[0.38433731314981423, 0.27872007475335486, 0.33694261207972276]

### Time Decay

In [28]:
df = pd.concat(
    [
        pb.footballdata.fetch_data("england", 2016, 0),
        pb.footballdata.fetch_data("england", 2017, 0),
        pb.footballdata.fetch_data("england", 2018, 0),
        pb.footballdata.fetch_data("england", 2019, 0),
        pb.footballdata.fetch_data("england", 2020, 0),
    ]
).reset_index()

In [29]:
weights = pb.poisson.dixon_coles_weights(df["Date"], 0.0018)

In [30]:
dc = pb.poisson.DixonColesGoalModel(
    df["FTHG"],
    df["FTAG"],
    df["HomeTeam"],
    df["AwayTeam"],
    weights,
)

dc.fit()

In [31]:
dc

Module: Penaltyblog

Model: Dixon and Coles

Number of parameters: 62
Log Likelihood: -1753.91
AIC: 3631.82

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.338                -1.11               
Aston Villa          1.148                -0.936              
Bournemouth          1.08                 -0.681              
Brighton             0.885                -0.991              
Burnley              0.888                -0.913              
Cardiff              0.782                -0.672              
Chelsea              1.365                -1.165              
Crystal Palace       0.955                -0.81               
Everton              1.074                -0.996              
Fulham               0.589                -0.86               
Huddersfield         0.414                -0.65               
Hull                 0.82                 -0.471              
Leeds      